# ACT-R Demo

- The model will be a mind that makes just one action
- `plan` to turn one card in `Memory`. 
- check that the `game` to be played is `Memory`. If so, take an action. 
- Assume that there are 10 cards that can be uncovered by pressing numbers 1,2,...9,0. 
- the `action` should be to `press a key` with one of the numbers.

## Goal buffer & Production rule

In [2]:
import pyactr as actr
import warnings
warnings.filterwarnings("ignore")

# ACT-R model
playing_memory = actr.ACTRModel()

Create two knowledge:
- The goal knowledge
- The production / rule knowledge

In [ ]:
#Creating a chunk the goal will carry 
# name chunk type and define slots the buffer carry
# chunk type - playgame
# slots - game , activity
actr.chunktype("playgame", "game, activity")

# token the goal buffer has at start of simulation
# chunk typename - playgame, 
# game - memory, 
# activity - none
# activity will be decided through the mind with production knowledge
initial_chunk = actr.makechunk(typename="playgame", game="memory")

# set goal buffer in model
goal = playing_memory.set_goal("goal")
goal.add(initial_chunk)

In [17]:
# current state of our goal - no activity
print(goal)

{playgame(activity= , game= memory)}


### Production rule 

- `=goal>` to `==>` is the IF statement, the chunk that must be in buffer for the rule to fire
- After `==>` is the `action` taken if the rule fires, which in this case is modify the buffer, from `None` to `presskey`

In [12]:
playing_memory.productionstring(name="startplaying", string="""
=goal>
isa      playgame
game     memory
activity None
==>
=goal>
isa playgame
activity presskey""")

{'=goal': playgame(activity= None, game= memory)}
==>
{'=goal': playgame(activity= presskey, game= )}

### Simulation 

The simulation output traces, which contain three things 
- time 
- ACT-R module
- Action 

In [13]:
simulation_game = playing_memory.simulation()
simulation_game.run()

(0, 'PROCEDURAL', 'CONFLICT RESOLUTION')
(0, 'PROCEDURAL', 'RULE SELECTED: startplaying')
(0.05, 'PROCEDURAL', 'RULE FIRED: startplaying')
(0.05, 'goal', 'MODIFIED')
(0.05, 'PROCEDURAL', 'CONFLICT RESOLUTION')
(0.05, 'PROCEDURAL', 'NO RULE FOUND')


In [18]:
# value of activity changed from none to presskey
print(goal)

{playgame(activity= , game= memory)}


## Add Motor module

### Production 2 

- let the mind modify the environment through the `motor` module. 
- The `motor` module can simulate a `key press` on a keyboard.
- IF the chunk type is playgame, and game- memory and activity - presskey
- Take action using the `motor` buffer, `manual` commanding to press a key 1
- Change the `activity` to `none` since it is taken by the motor module

In [21]:
playing_memory.productionstring(name="presskey", string="""
=goal>
isa  playgame
game memory
activity presskey
==>
+manual>
isa _manual
cmd press_key
key 1
=goal>
isa playgame
activity None""")

{'=goal': playgame(activity= presskey, game= memory)}
==>
{'+manual': _manual(cmd= press_key, key= 1), '=goal': playgame(activity= None, game= )}

In [22]:
# Now to start the simulation we have to add the chunk in the goal buffer
# the chunk is modified with first production rule which is game-memory, activity-presskey
goal.add(initial_chunk)
simulation_game = playing_memory.simulation()
simulation_game.run()

(0, 'PROCEDURAL', 'CONFLICT RESOLUTION')
(0, 'PROCEDURAL', 'RULE SELECTED: startplaying')
(0.05, 'PROCEDURAL', 'RULE FIRED: startplaying')
(0.05, 'goal', 'MODIFIED')
(0.05, 'PROCEDURAL', 'CONFLICT RESOLUTION')
(0.05, 'PROCEDURAL', 'RULE SELECTED: presskey')
(0.1, 'PROCEDURAL', 'RULE FIRED: presskey')
(0.1, 'goal', 'MODIFIED')
(0.1, 'manual', 'COMMAND: press_key')
(0.1, 'PROCEDURAL', 'CONFLICT RESOLUTION')
(0.1, 'PROCEDURAL', 'RULE SELECTED: startplaying')
(0.15, 'PROCEDURAL', 'RULE FIRED: startplaying')
(0.15, 'goal', 'MODIFIED')
(0.15, 'PROCEDURAL', 'CONFLICT RESOLUTION')
(0.15, 'PROCEDURAL', 'RULE SELECTED: presskey')
(0.2, 'PROCEDURAL', 'RULE FIRED: presskey')
(0.2, 'goal', 'MODIFIED')
(0.2, 'manual', 'COMMAND: press_key')
(0.2, 'PROCEDURAL', 'CONFLICT RESOLUTION')
(0.2, 'PROCEDURAL', 'RULE SELECTED: startplaying')
(0.25, 'PROCEDURAL', 'RULE FIRED: startplaying')
(0.25, 'goal', 'MODIFIED')
(0.25, 'PROCEDURAL', 'CONFLICT RESOLUTION')
(0.25, 'PROCEDURAL', 'RULE SELECTED: presskey')
(0

In [23]:
# value of activity after runnig the production
print(goal)

{playgame(activity= presskey, game= memory)}


- The two rules keep looping  quickly that the model never gets a chance to actually press the key using manual buffer.

- This is because the `startplaying` rule fires without waiting for the `manual` buffer `press the key `
- We have to modify the first production, `startplaying` to only fire if the `manual` buffer is not trying to `press` a key. 
- We will do this by adding a query on the `state` of manual buffer being `free` or is not carrying action. 

In [24]:
playing_memory.productionstring(name="startplaying", string="""
=goal>
isa  playgame
game memory
activity None
?manual>
state free
==>
=goal>
isa playgame
activity presskey""")

{'=goal': playgame(activity= None, game= memory), '?manual': {'state': 'free'}}
==>
{'=goal': playgame(activity= presskey, game= )}

In [25]:
goal.add(initial_chunk)
simulation_game = playing_memory.simulation()
simulation_game.run()

(0, 'PROCEDURAL', 'CONFLICT RESOLUTION')
(0, 'PROCEDURAL', 'RULE SELECTED: startplaying')
(0.05, 'PROCEDURAL', 'RULE FIRED: startplaying')
(0.05, 'goal', 'MODIFIED')
(0.05, 'PROCEDURAL', 'CONFLICT RESOLUTION')
(0.05, 'PROCEDURAL', 'RULE SELECTED: presskey')
(0.1, 'PROCEDURAL', 'RULE FIRED: presskey')
(0.1, 'goal', 'MODIFIED')
(0.1, 'manual', 'COMMAND: press_key')
(0.1, 'PROCEDURAL', 'CONFLICT RESOLUTION')
(0.1, 'PROCEDURAL', 'NO RULE FOUND')
(0.35, 'manual', 'PREPARATION COMPLETE')
(0.35, 'PROCEDURAL', 'CONFLICT RESOLUTION')
(0.35, 'PROCEDURAL', 'NO RULE FOUND')
(0.4, 'manual', 'INITIATION COMPLETE')
(0.4, 'PROCEDURAL', 'CONFLICT RESOLUTION')
(0.4, 'PROCEDURAL', 'NO RULE FOUND')
(0.5, 'manual', 'KEY PRESSED: 1')
(0.5, 'PROCEDURAL', 'CONFLICT RESOLUTION')
(0.5, 'PROCEDURAL', 'NO RULE FOUND')
(0.65, 'manual', 'MOVEMENT FINISHED')
(0.65, 'PROCEDURAL', 'CONFLICT RESOLUTION')
(0.65, 'PROCEDURAL', 'RULE SELECTED: startplaying')
(0.7, 'PROCEDURAL', 'RULE FIRED: startplaying')
(0.7, 'goal', 'M

- Now after the command `press_key` from the manual buffer at `0.1 sec` the following things will happen 
    - Preparation complete `0.35 sec`
    - initiation complete `0.4 sec`
    - press the key 1  `0.5 sec`
    - movement finished `0.65 sec`
- Through out this time no rules are fired till the manual buffer state finished the action and become free
- After that the `presskey` rule finished, the goal activity become back to None so the `startplaying` rule can be fired again 

In [ ]:
# value of activity after runnig the productions
print(goal)

{playgame(activity= None, game= memory)}
